## Problem Statement

In supply chain management, forecasting demand and optimizing inventory are vital for maintaining operational efficiency and customer satisfaction. As a data analyst, I applied time series analysis to predict future sales for Product P1 and calculate safety stock levels to manage demand variability. The goal was to determine the reorder point that triggers inventory replenishment, ensuring that stock levels are optimal—neither overstocked nor understocked. Additionally, I utilized the Economic Order Quantity (EOQ) model to identify the most cost-efficient order quantity, reducing inventory costs. Finally, I established the reorder cycle, defining the time intervals at which orders should be placed to maintain smooth inventory operations.

## Process Overview:
1. Preprocess the dataset.
2. Analyze demand and inventory trends.
3. Test for stationarity (ADF Test).
4. Model demand using SARIMAX and forecast.
5. Optimize inventory with Newsvendor formula and reorder point.
6. Calculate safety stock and total cost.
7. Develop inventory management strategy.



## Data:

The dataset contains daily information for a product's demand and inventory levels. Below is a brief overview of the columns:

**Date**: The date corresponding to the entry.
**Product_ID**: The unique identifier for the product (e.g., P1).
**Demand**: The number of units demanded on that particular date.
**Inventory**: The number of units available in stock on that date.

**Data Source**: [demand_forecasting](<../../OneDrive/Documents/Project 365/Python/Datasets/demand_inventory.csv>)




## Table of Contents:

1.	Import necessary libraries and load the dataset.
2.	Preprocess the dataset by removing unnecessary columns.
3.	Visualize demand and inventory over time.
4.	Perform stationarity test (ADF Test).
5.	Apply differencing to make the demand data stationary.
6.	Plot ACF and PACF for ARIMA/SARIMA parameter selection.
7.	Fit SARIMAX model and forecast demand.
8.	Generate future demand predictions for the next 10 days.
9.	Calculate the optimal order quantity using the Newsvendor formula.
10.	Compute reorder point considering lead time and variability.
11.	Determine safety stock to account for demand uncertainty.
12.	Calculate total cost (holding + stockout cost).
13.	Conclusion



### 1. Importing Libraries and Loading the Dataset

In [ ]:
!pip install plotly

In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
import statsmodels.api as sm 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX


### 2. Preprocessing the Dataset

In [ ]:
df = pd.read_csv(r'C:\Users\ajayk\Downloads\Demand-Forecasting-and-Inventory-Optimization\demand_inventory.csv')

df = df.drop(columns=['Unnamed: 0'])

*Here, we load the dataset and remove the unnecessary index column (Unnamed: 0) that was automatically added during data export.*

### Visualizing demand and inventory over time.

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(df['Date'],df['Demand'],color='green', marker='o', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.title('Demand over time')
plt.grid(True)
plt.xticks(rotation=90)
plt.show()

In [ ]:
import plotly.express as px

fig_demand = px.line(df, x='Date',
                     y='Demand',
                     title='Demand Over Time')
fig_demand.show()

In [ ]:

fig_inventory = px.line(df, x='Date',
                     y='Inventory',
                     title='Inventory Over Time')
fig_inventory.show()

*These visualizations help identify trends in demand and inventory over time. The Demand and Inventory time series are plotted for clear visualization of their respective patterns.*

## **Demand Forecasting**

### 4. Performing Stationarity Test (ADF Test)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result=adfuller(df)
    print('ADF STATS : {}'.format(result[0]))
    print('p-value : {}'.format(result[1]))
    if result[1] >= 0.05:
        print ("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary")
        

adf_test(df["Demand"])  

*The Augmented Dickey-Fuller (ADF) test is applied to check the stationarity of the demand data. If the p-value is greater than 0.05, the null hypothesis of non-stationarity is rejected, indicating that the data is stationary.*

### 5. Differencing the Demand Data to Make it Stationary

In [ ]:

df['1st_diff_demand'] = df['Demand'].diff().dropna()
df 

*To make the data stationary, we performed differencing on the demand data. This operation subtracts the current value from the previous value to remove trends in the series.*

In [ ]:
fig_demand = px.line(df, x='Date',
                     y='1st_diff_demand',
                     title='Demand Over Time')
fig_demand.show()

### 6. Plotting ACF and PACF for ARIMA/SARIMA Parameter Selection

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

df['Date'] = pd.to_datetime(df['Date'],
                                     format='%Y/%m/%d')
time_series = df.set_index('Date')['Demand']

differenced_series = time_series.diff().dropna()

# Plot ACF and PACF of differenced time series
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(differenced_series, ax=axes[0])
plot_pacf(differenced_series, ax=axes[1])
plt.show()

*We plotted the AutoCorrelation Function (ACF) and Partial AutoCorrelation Function (PACF) to identify the order of the AR (AutoRegressive) and MA (Moving Average) components for the SARIMA model. Based on these plots, we chose parameters p=3 and q=1 for the SARIMAX model.*

### 7. Fitting the SARIMAX Model and Forecasting Demand

In [ ]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 2) #2 because the data contains a time period of 2 months only
model = SARIMAX(time_series, order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)



*The SARIMAX model is fitted using the identified parameters (order = (1, 1, 1) and seasonal_order = (1, 1, 1, 2)). This model accounts for both trend and seasonality in the demand data.*

### 8. Creating future demand predictions for the next 10 days.

In [ ]:
future_steps = 10
predictions = model_fit.predict(len(time_series), len(time_series) + future_steps - 1)
predictions = predictions.astype(int)
print(predictions)

## **Inventory Optimization**

*The goal is to optimize inventory based on the forecasted demand for the next ten days.*


In [ ]:
# Create date indices for the future predictions
future_dates = pd.date_range(start=time_series.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='D')

# Create a pandas Series with the predicted values and date indices
forecasted_demand = pd.Series(predictions, index=future_dates)
print(forecasted_demand)  

In [ ]:
# Initial inventory level
initial_inventory = 5500

# Lead time (number of days it takes to replenish inventory) 
lead_time = 2 # it's different for every business, 2 days as an example

# Service level (probability of not stocking out)
service_level = 0.95 # it's different for every business, 0.95 is an example




***The Economic Order Quantity (EOQ)** or **optimal order quantity** is the amount of stock that should be ordered to minimize the total costs associated with ordering and holding inventory. This is calculated based on the Newsvendor model, taking into account factors such as:*

*Q∗ =μ+z⋅σ*

*Q∗ is the optimal order quantity. or The Economic Order Quantity*

*𝜇 - μ is the mean (average) of the forecasted demand.*

*𝜎 - σ is the standard deviation of the forecasted demand.*

*𝑧 - z is the z-score corresponding to the desired service level, obtained        using norm.ppf(service_level).*

***Service Level Approach**: By using the z-score, the formula adjusts the order quantity to achieve the desired service level. The z-score tells us how many standard deviations above the mean demand we need to go to ensure that we meet the demand with the given probability (service level).*



### 9. Calculate the optimal order quantity using the Newsvendor formula

In [ ]:
from scipy.stats import norm

forecasted_demand_mean = forecasted_demand.mean()
forecasted_demand_std = forecasted_demand.std()

# Calculate the z-score for the desired service level i.e 0.95
z = norm.ppf(service_level)

# Calculate the optimal order quantity using the Newsvendor formula
order_quantity = np.ceil(forecasted_demand_mean + z * forecasted_demand_std).astype(int)

print(f'Optimal Order Quantity: {order_quantity}')

### **Explaination**

***Normal Distribution:***

*The Newsvendor problem assumes that the demand follows a normal distribution. This is a common assumption because many natural phenomena tend to follow a normal distribution due to the Central Limit Theorem.*

***Service Level***:

*The service level represents the probability that the demand will not exceed the stock level. For example, a 95% service level means that there is a 95% chance that the demand will be met without stocking out.*


***Z-Score:***

The z-score is a statistical measure that represents the number of standard deviations a data point is from the mean. In this case, it represents how far the optimal stock level is from the mean demand, in terms of standard deviations.*

***Percent Point Function (ppf):***

*The ppf function is the inverse of the cumulative distribution function (CDF). While the CDF gives the probability that a value is less than or equal to a certain value, the ppf function gives the value that corresponds to a given probability.*


*By using norm.ppf(service_level), we obtain the z-score that corresponds to the desired service level. For instance, for a service level of 95%, norm.ppf(0.95) gives us the z-score that corresponds to the 95th percentile of the normal distribution.*

### 10.	Compute reorder point considering lead time and variability.

In [ ]:
std_lead_time_demand = forecasted_demand_std * np.sqrt(lead_time)

# Computing reorder point considering lead time and variability.
reorder_point = forecasted_demand_mean * lead_time + z * std_lead_time_demand
print(f'Reorder Point: {reorder_point:.2f} units')

### Parameters Summary:

***EOQ (Economic Order Quantity)** or **Optimal order quantity:***
*EOQ focuses on optimizing the order size to minimize costs.*

***ROP (Reorder Point):*** 
*ROP focuses on the timing of placing new orders to prevent stockouts.*


***EOQ - 133 units:***
*This is the amount you should order each time you place an order to minimize the total inventory costs while achieving the desired service level. It considers the average demand and variability, ensuring that you order enough to meet demand without overstocking.*

***ROP - 258.15 units:*** 
*This is the inventory level at which you should place a new order to ensure you don't run out of stock before the new order arrives. It considers the lead time and variability in demand during this period.*


*Both metrics should work together. When your inventory level hits the ROP, you place an order of the EOQ to replenish your stock. In this case, if the inventory hits 258 units, you place an order of 133 units to replenish.*


### 11. Determine safety stock to account for demand uncertainty.

In [ ]:
safety_stock = reorder_point - forecasted_demand.mean() * lead_time
print(f'safety_stock: {reorder_point:.2f} units')


#where  forecasted_demand.mean() * lead_time is avg_demand_lead_time 

*Safety stock is the additional inventory kept on hand to account for uncertainties in demand and supply. It acts as a buffer against unexpected variations in demand or lead time. In this case, a safety stock of **258 units** has been calculated, which helps ensure that there’s enough inventory to cover potential fluctuations in demand or lead time.*

### 12.	Calculate total cost (holding + stockout cost).

In [ ]:
holding_cost = 0.1  # it's different for every business, 0.1 is an example
stockout_cost = 10  # # it's different for every business, 10 is an example
total_holding_cost = holding_cost * (initial_inventory + 0.5 * order_quantity)
total_stockout_cost = stockout_cost * np.maximum(0, forecasted_demand.mean() * lead_time - initial_inventory)

total_cost = total_holding_cost + total_stockout_cost

print("Total Cost:", total_cost)


*The total cost represents the combined costs associated with inventory management. In this case, the total cost has been calculated as approximately **561.80** units based on the order quantity, reorder point, safety stock, and associated costs.*

### **Conclusion**

Demand Forecasting involves predicting customer order quantities and patterns, which is vital for businesses to allocate resources effectively, manage inventory, and plan production. Inventory Optimization seeks to maintain an optimal balance—ensuring enough stock is available to meet demand, while avoiding excess inventory that ties up capital and storage space.

In conclusion, this analysis offers a robust approach to demand forecasting and inventory optimization for Product P1. By utilizing time series analysis, the SARIMAX model successfully forecasts future demand. Simultaneously, the Economic Order Quantity (EOQ) and reorder point calculations facilitate efficient inventory replenishment. The safety stock metric further mitigates risks by buffering against fluctuations in demand and supply. When combined with inventory cost analysis, these insights form a comprehensive strategy that minimizes costs while maintaining product availability. This integrated approach enhances overall supply chain efficiency and empowers data-driven decision-making, contributing to operational success.